In [1]:
import pickle
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
import numpy as np
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, f1_score
from sklearn.model_selection import cross_validate, KFold
import statsmodels.api as sm
from random import shuffle
import plotly.plotly as py
import plotly.graph_objs as go
#py.sign_in('username', 'key')
from sklearn import datasets

/home/velocci/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# Python 2 code:
#import operator
#import copy
#from collections import defaultdict
#import math
#from sklearn.neighbors import KNeighborsClassifier
#import matplotlib.pyplot as plt
#%matplotlib inline
#from sklearn.learning_curve import learning_curve
# from sklearn.naive_bayes import GaussianNB
# from sklearn.svm import SVC
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# from sklearn.cross_validation import cross_val_score, train_test_split, KFold, StratifiedKFold
# from sklearn.pipeline import Pipeline
# from sklearn.grid_search import GridSearchCV

In [3]:
file = open("buzzfeed.pickle",'rb')
buzzfeed = pickle.load(file)
file = open("viralnova.pickle", 'rb')
viralnova = pickle.load(file)
file = open("headlines_2016_2014.pickle", 'rb')
headlines_2016_2014 = pickle.load(file, encoding='bytes')
file = open("headlines_wsj_2016.pickle", 'rb')
headlines_wsj_2016 = pickle.load(file)

buzzfeed = buzzfeed[0:19638]
headlines_wsj_2016 = headlines_wsj_2016[0:15500]
headlines_2016_2014 = headlines_2016_2014[0:15500]

headlines = []
headlines.extend(headlines_wsj_2016)
headlines.extend(headlines_2016_2014)

all_articles = []
all_articles.extend(buzzfeed)
all_articles.extend(viralnova)
all_articles.extend(headlines)

In [4]:
# split corpus into train and test
# do tfidf.fit on train
# combine train and test and do tfidf.transform on entire corpus

article_df = pd.DataFrame(all_articles)

In [5]:
article_df['y'] = 0

In [6]:
article_df.loc[0:30999,"y"] = 1
#article_df['y'].loc[0:30999] = 1
#y = article_df[['y']]

In [7]:
train=article_df.sample(frac=0.8,random_state=200)
test=article_df.drop(train.index)

In [8]:
y_train = train['y']

In [9]:
y_test = test['y']

In [10]:
del train['y']
del test['y']

In [11]:
train_corpus = []
for title in train[0]:
    train_corpus.append(title)
test_corpus = []
for title in test[0]:
    test_corpus.append(title)

In [12]:
type(train)

pandas.core.frame.DataFrame

In [13]:
# This code is needed to form new columns 'first_odd' below.  '1' if title begins with an odd number, '0' otherwise.
# first_num_train = []
# first_num_test = []
# first_odd_train = []
# first_odd_test = []
# numbers = [str(num) for num in range(1,51)]
# odds = [str(num) for num in range(1,51,2)]
# for headline in train_corpus:
#     if headline.split()[0] in numbers:
#         first_num_train.append(1)
#     else:
#         first_num_train.append(0)
#     if headline.split()[0] in odds:
#         first_odd_train.append(1)
#     else:
#         first_odd_train.append(0)
# for headline in test_corpus:
#     if headline.split()[0] in numbers:
#         first_num_test.append(1)
#     else:
#         first_num_test.append(0)
#     if headline.split()[0] in odds:
#         first_odd_test.append(1)
#     else:
#         first_odd_test.append(0)

In [14]:
stop_words = ['i','me','my','myself','we','our','ours','ourselves','yours','yourself','yourselves','he','him','his','himself',
    'she','her','hers','herself','it','its','itself','they','them','their','theirs','themselves','who','whom','this',
    'that','these','those','am','is','are','was','were','be','been','being','have','has','had','having','do','does','did',
    'doing','a','an','the','and','but','if','or','because','as','until','while','of','at','by','for','with','about','against',
    'between','into','through','during','before','after','above','below','to','from','up','down','in','out','on','off','over','under',
    'again','further','then','once','here','there','when','where','why','how','all','any','both','each','few','more','most',
    'other','some','such','no','nor','not','only','own','same','so','than','too','very','s','t','can','will','just','don',
    'should','now','d','ll','m','o','re','ve','y','ain','aren','couldn','didn','doesn','hadn','hasn','haven','isn','ma','mightn',
    'mustn','needn','shan','shouldn','wasn','weren','won','wouldn','said']

more_stop_words = [str(i) for i in range(1,1001)]
normal_stops = ['you', 'your', 'which', 'what']

all_stops = stop_words + more_stop_words# + normal_stops

tfidf_vectorizer = TfidfVectorizer(analyzer='word',
                                  ngram_range=(1, 3), stop_words=all_stops,
                                  min_df=3, max_df=1.0, max_features=None, strip_accents='unicode', token_pattern=r'\w{1,}',
                                  sublinear_tf=True)
tfidf_fit = tfidf_vectorizer.fit(train_corpus)

tfidf_transform = tfidf_fit.transform(train_corpus + test_corpus)

df_proper = pd.DataFrame(tfidf_transform.todense(), columns=[tfidf_vectorizer.get_feature_names()])

train = df_proper.loc[0:49599]
test = df_proper.loc[49600:61999]

In [15]:
# Not working for some reason
#train.loc[:, 'first_odd'] = first_odd_train
#test.loc[:,"first_odd"] = first_odd_test

In [16]:
# #train['first_num'] = train['first_num'].astype('category')
# #train['first_odd'] = train['first_odd'].astype('category')
# #test['first_num'] = test['first_num'].astype('category')
# #test['first_odd'] = test['first_odd'].astype('category')

# Not needed if I don't run the cell above this one
# train = pd.get_dummies(train, columns=['first_odd'])#, 'first_num'])
# test = pd.get_dummies(test, columns=['first_odd'])#, 'first_num'])

In [36]:
model = LogisticRegression()

In [37]:
def model_info(train, test):
    model.fit(train, y_train)
    digit_prec=2
    truthvec = y_test
    try:
        model.predict_proba
        scorevec = model.predict_proba(test)[:,1]
        threshvec = np.unique(np.round(scorevec,digit_prec))
        numthresh = len(threshvec)
        tpvec = np.zeros(numthresh)
        fpvec = np.zeros(numthresh)
        fnvec = np.zeros(numthresh)
        for i in range(numthresh):
            thresh = threshvec[i]
            tpvec[i] = sum(truthvec[scorevec>=thresh])
            fpvec[i] = sum(1-truthvec[scorevec>=thresh])
            fnvec[i] = sum(truthvec[scorevec<thresh])
        recallvec = tpvec/(tpvec + fnvec)
        precisionvec = tpvec/(tpvec + fpvec)
        f1vec = 2*precisionvec*recallvec/(precisionvec + recallvec)
        f1_x = list(zip(threshvec, f1vec))
        f1_index = [i for i, v in enumerate(f1_x) if v[0] == 0.5][0]
        f1 = f1_x[f1_index][1]
        print('f1-score ', f1)

    except:
        scorevec = model.predict(test)
        print('F1: ', f1_score(y_test, scorevec))
    score = roc_auc_score(y_test, scorevec)
    print('AUC: ', score)
    rec_x = list(zip(threshvec, recallvec))
    rec_index = [i for i, v in enumerate(rec_x) if v[0] == 0.5][0]
    recall = rec_x[rec_index][1]
    print('recall ', recall)

    prec_x = list(zip(threshvec, precisionvec))
    prec_index = [i for i, v in enumerate(prec_x) if v[0] == 0.5][0]
    precision = prec_x[prec_index][1]
    print('precision ', precision)

    y_pred = model.predict(test)
    print('accuracy', accuracy_score(truthvec,y_pred))
    col_keep = []
    col_reg = []
    try:
        model.feature_importances_
        features = list(zip(model.feature_importances_, train.columns))
        features.sort(key = lambda t: t[0], reverse=True)
        for i in range(len(features)):
            if features[i][0] != 0:
                col_keep.append(features[i][1])
                print('Importance: ', features[i])
            else:
                col_reg.append(features[i][1])
    except:
        print('Intercept: ', model.intercept_)
        zipped = sorted(list(zip(model.coef_[0], train.columns)), key= lambda t: abs(t[0]), reverse=True)
        for i in range(len(zipped)):
            if zipped[i][0] != 0:
                col_keep.append(zipped[i][1])
                print('Coefficient: ',zipped[i])
            else:
                col_reg.append(zipped[i][1])
    return model, col_keep, col_reg

In [38]:
model, col_keep, col_reg = model_info(train, test)

/home/velocci/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning:

invalid value encountered in true_divide



f1-score  0.944272948822
AUC:  0.985138986084
recall  0.93290529695
precision  0.955921052632
accuracy 0.944677419355
Intercept:  [-1.02814645]
Coefficient:  (16.079236814865208, ('you',))
Coefficient:  (8.8978017293162051, ('your',))
Coefficient:  (7.6797583131002289, ('people',))
Coefficient:  (6.3950267322387235, ('which',))
Coefficient:  (6.3580417937654472, ('dog',))
Coefficient:  (-6.1108456295210747, ('review',))
Coefficient:  (5.8859232200375384, ('everyone',))
Coefficient:  (5.8710689430528813, ('hilarious',))
Coefficient:  (5.8671980698652764, ('times',))
Coefficient:  (5.8254395173103291, ('adorable',))
Coefficient:  (5.2669170241354513, ('things',))
Coefficient:  (5.2294354824393583, ('beautiful',))
Coefficient:  (5.1992243602233561, ('actually',))
Coefficient:  (5.1823853938949922, ('amazing',))
Coefficient:  (5.0457874145229855, ('awesome',))
Coefficient:  (5.0412399315551015, ('something',))
Coefficient:  (4.8081646973910273, ('lgbt',))
Coefficient:  (4.806602793050792, 

Coefficient:  (2.0789410622911282, ('ghost',))
Coefficient:  (2.0777882565818264, ('genius',))
Coefficient:  (-2.0768940612858504, ('plan',))
Coefficient:  (2.0691156090775915, ('giant',))
Coefficient:  (-2.0676758244432913, ('economy',))
Coefficient:  (2.0637536053644596, ('taylor swift',))
Coefficient:  (2.0609241561376561, ('simple',))
Coefficient:  (2.0595480848791317, ('instead',))
Coefficient:  (2.0547372118341913, ('britney',))
Coefficient:  (2.0427073822189468, ('words',))
Coefficient:  (-2.0409254552856084, ('ahead',))
Coefficient:  (2.0397414350096961, ('posted',))
Coefficient:  (-2.0360112138328277, ('brexit',))
Coefficient:  (-2.0342808829758652, ('revenue',))
Coefficient:  (2.027823396210918, ('mouthwatering',))
Coefficient:  (-2.0143736477923984, ('crash',))
Coefficient:  (2.0141377733352761, ('happen',))
Coefficient:  (2.0135940845965838, ('pets',))
Coefficient:  (2.0105996447721202, ('internet',))
Coefficient:  (2.0069572397520372, ('hair',))
Coefficient:  (2.0054648240

Coefficient:  (0.82731701887439379, ('luke',))
Coefficient:  (0.82632066593911757, ('goth',))
Coefficient:  (0.82598313895981557, ('typical',))
Coefficient:  (-0.82583855222564029, ('taliban',))
Coefficient:  (0.82581708576021229, ('nap',))
Coefficient:  (0.82571316065829448, ('shut',))
Coefficient:  (0.82560853221517094, ('committed',))
Coefficient:  (0.8255484543610615, ('you never',))
Coefficient:  (-0.8252239732298462, ('advance',))
Coefficient:  (0.82518709066787077, ('parks rec',))
Coefficient:  (0.82518709066787077, ('rec',))
Coefficient:  (0.82483295074577612, ('next',))
Coefficient:  (0.82440522388520432, ('ariana',))
Coefficient:  (0.82437389981349296, ('grande',))
Coefficient:  (0.82395955255632225, ('fantastic',))
Coefficient:  (0.82369294461874876, ('presented',))
Coefficient:  (-0.8235171778761794, ('scandal',))
Coefficient:  (0.82332216798047908, ('every year',))
Coefficient:  (0.82299117375153941, ('flew',))
Coefficient:  (-0.8229859979150187, ('lawsuit',))
Coefficient:

Coefficient:  (-0.55676200112704888, ('nomination',))
Coefficient:  (-0.55672740210733818, ('proposes',))
Coefficient:  (-0.5566204720797211, ('independence',))
Coefficient:  (0.55654357581377734, ('say hello',))
Coefficient:  (-0.55631686220511645, ('wall',))
Coefficient:  (-0.5563141178354537, ('ftc',))
Coefficient:  (-0.55613148437440107, ('balance',))
Coefficient:  (-0.55565294500274354, ('along',))
Coefficient:  (-0.55565156426209283, ('traders',))
Coefficient:  (-0.55543385069252948, ('phil',))
Coefficient:  (0.5553262854023221, ('surgeon',))
Coefficient:  (0.55527228016548313, ('perfection',))
Coefficient:  (0.55504513574867198, ('toilet',))
Coefficient:  (0.55475551658706845, ('brutally honest',))
Coefficient:  (0.55468557377589855, ('viking',))
Coefficient:  (-0.5545397620243816, ('wider',))
Coefficient:  (0.55426396244024756, ('myths',))
Coefficient:  (-0.55423451955954828, ('acquire',))
Coefficient:  (-0.5541769104518216, ('virus',))
Coefficient:  (0.5541509296984285, ('gwen

...

Coefficient:  (-0.096510729960758337, ('you winter',))
Coefficient:  (0.096508901572897501, ('unsee',))
Coefficient:  (-0.096499064188437789, ('like children',))
Coefficient:  (0.096496962570460532, ('rolling laughter',))
Coefficient:  (0.096484901119643085, ('signs your',))
Coefficient:  (-0.096482553575123378, ('president says',))
Coefficient:  (0.096473874633228235, ('time find',))
Coefficient:  (-0.096447248572051308, ('turkey says',))
Coefficient:  (-0.096415916150245376, ('chip',))
Coefficient:  (-0.096414421429991098, ('2010',))
Coefficient:  (0.096391968366311351, ('letting',))
Coefficient:  (0.09638995227439015, ('world largest',))
Coefficient:  (-0.096383043788697165, ('bumgarner giants',))
Coefficient:  (0.096359152358495612, ('stories people',))
Coefficient:  (0.096344147047558074, ('your coffee',))
Coefficient:  (0.09634403530558984, ('cait',))
Coefficient:  (-0.096331010387613283, ('rays top',))
Coefficient:  (-0.096331010387613283, ('rays top twins',))
Coefficient:  (0.0

Coefficient:  (0.075095893871404806, ('monica chandler',))
Coefficient:  (0.075076058692370717, ('spreading',))
Coefficient:  (-0.07505145084073217, ('deal talks',))
Coefficient:  (-0.07501776844042872, ('digital ads',))
Coefficient:  (-0.075000385113172016, ('brexit could',))
Coefficient:  (-0.074993057146122991, ('downgraded',))
Coefficient:  (0.074964778710178134, ('number you',))
Coefficient:  (0.074946056279951448, ('you remember first',))
Coefficient:  (0.074938978224476649, ('want eat',))
Coefficient:  (0.074925597930408303, ('what job',))
Coefficient:  (0.074906746983772796, ('make you melt',))
Coefficient:  (0.074885282170222459, ('totally adorable',))
Coefficient:  (-0.074878830381806122, ('illness',))
Coefficient:  (0.074870388100567634, ('stigma',))
Coefficient:  (0.074843934472569054, ('holiday gifts',))
Coefficient:  (0.074816982891896047, ('hiding something',))
Coefficient:  (-0.074809421840178636, ('huawei',))
Coefficient:  (-0.074803208779068681, ('healthier',))
Coeffi

Coefficient:  (-0.054982663018287775, ('cameron appeals',))
Coefficient:  (-0.054976190364469053, ('mellon',))
Coefficient:  (-0.054964961056407503, ('tranquil',))
Coefficient:  (-0.054942559338989595, ('watchdog warns',))
Coefficient:  (0.054920061345717607, ('ominous',))
Coefficient:  (-0.05488959706184731, ('u eu',))
Coefficient:  (0.054884429090510264, ('looks gross',))
Coefficient:  (-0.054873245734410388, ('second quarter 2016',))
Coefficient:  (0.054865729652603772, ('you little',))
Coefficient:  (0.054848426551442425, ('character you hook',))
Coefficient:  (-0.054829891251656937, ('defeating',))
Coefficient:  (0.054825411568928686, ('greyhound',))
Coefficient:  (0.054809155279451481, ('little real',))
Coefficient:  (-0.054785378004903892, ('wrongful',))
Coefficient:  (-0.054780293370698299, ('ukraine conflict',))
Coefficient:  (-0.054778167928008319, ('u government bond',))
Coefficient:  (-0.054751763082229933, ('new board',))
Coefficient:  (0.05472759835708671, ('beer you',))


Coefficient:  (0.034922364263235341, ('you thanksgiving',))
Coefficient:  (0.034895162252799934, ('halloween episode',))
Coefficient:  (0.034886986754755669, ('you fantasize',))
Coefficient:  (0.034847790634775784, ('false advertising',))
Coefficient:  (0.034842901286919796, ('matches your favorite',))
Coefficient:  (0.034833498124672325, ('your life easier',))
Coefficient:  (-0.03479942465557799, ('turns',))
Coefficient:  (-0.034779491266043044, ('clash police',))
Coefficient:  (-0.03476418538519542, ('sailor',))
Coefficient:  (0.034752038748100775, ('emoji choices',))
Coefficient:  (0.034752038748100775, ('your emoji choices',))
Coefficient:  (-0.034743814828082864, ('catcher',))
Coefficient:  (0.034729184011325687, ('totally make',))
Coefficient:  (-0.03472347254113417, ('montreal',))
Coefficient:  (0.034722010473240784, ('adorable animal',))
Coefficient:  (-0.034696873637740287, ('albums',))
Coefficient:  (0.03469509344461752, ('since 2007',))
Coefficient:  (0.034694218478315865, (

Coefficient:  (0.016982148054465674, ('your crush based',))
Coefficient:  (0.016974198523544034, ('manny pacquiao',))
Coefficient:  (0.016967389493138481, ('feathers',))
Coefficient:  (0.016913040525861164, ('definitely never',))
Coefficient:  (0.016913040525861164, ('you definitely never',))
Coefficient:  (0.016911142549041795, ('ruled',))
Coefficient:  (0.016905218016045678, ('need see what',))
Coefficient:  (0.016886268268343371, ('boyfriend based',))
Coefficient:  (0.016829494936106991, ('forms',))
Coefficient:  (0.016805675598855138, ('you found',))
Coefficient:  (-0.016795110688945195, ('china zte',))
Coefficient:  (0.016756143701902471, ('treasure hunt',))
Coefficient:  (0.016741469463932544, ('ribs',))
Coefficient:  (-0.016737631413123684, ('secretary says',))
Coefficient:  (0.016720241535773794, ('nixed',))
Coefficient:  (0.01671265862578545, ('need you love',))
Coefficient:  (-0.016703447242490888, ('stale',))
Coefficient:  (0.016702129427991638, ('need see based',))
Coeffici

In [ ]:
# first_odd = []
# odds = [str(num) for num in range(1,51,2)]
# foo = tfidf_fit.transform([text])
# if text.split()[0] in odds:
#     first_odd.append(1)
# else:
#     first_odd.append(0)
# df = pd.DataFrame(foo.todense(), columns=[tfidf_vectorizer.get_feature_names()])

In [ ]:
# df['first_odd'] = first_odd
# df = pd.get_dummies(df, columns=['first_odd'])#, 'first_num'])

In [ ]:
# if 'first_odd_0' in df.columns:
#     df['first_odd_1'] = 0.0
# else:
#     df['first_odd_0'] = 0.0

In [28]:
#probs = model.predict_proba(df.iloc[0])[0]

In [ ]:
#print probs

In [30]:
#text = test_corpus[1]
#test3 = tfidf_fit.transform([text])
model.predict_proba(test.iloc[0])[0]
#print test3[0][0]

In [ ]:
import statsmodels.api as sm

In [ ]:
len(col_keep)

In [ ]:
train.shape

In [ ]:
train2 = sm.add_constant(train, prepend=False)
y_train = list(y_train)
logit = sm.Logit(y_train, train2.drop(col_reg, axis=1))
result = logit.fit()
print result.summary()

In [ ]:
len(col_keep)

In [ ]:
train_reg = train.drop(col_reg, axis=1)
test_reg = test.drop(col_reg, axis=1)

In [ ]:
model = ensemble.RandomForestClassifier(n_estimators=1000, max_depth=3, n_jobs=-1)

In [ ]:
model, col_keep, col_reg = model_info(train_reg, test_reg)

In [ ]:
model = ensemble.GradientBoostingClassifier(n_estimators=1000, max_depth=3)

In [ ]:
len(col_keep)
train_reg = train.drop(col_reg, axis=1)
test_reg = test.drop(col_reg, axis=1)

In [ ]:
model, col_keep, col_reg = model_info(train_reg, test_reg)

In [ ]:
rec_index = [i for i, v in enumerate(rec_x) if v[0] == 0.5][0]
recall = rec_x[rec_index][1]
print recall

prec_x = list(zip(threshvec, precisionvec))
prec_index = [i for i, v in enumerate(prec_x) if v[0] == 0.5][0]
precision = prec_x[prec_index][1]
print precision

y_pred = model.predict(test)
print(accuracy_score(truthvec,y_pred))

In [ ]:
headline8217 = [headline for headline in all_articles if '8217' in headline]

In [ ]:
headline8217

In [ ]:
def normalize_df(df):
    df_norm = df.copy()
    for i in df.columns:
        mean = df[i].mean()
        std = df[i].std()
        df_norm[i] = df_norm[i].apply(lambda x: (x - mean)/std)
    return df_norm

In [ ]:
df_proper.shape

In [ ]:
#df_proper_normalized = normalize_df(df_proper)
# pca = decomposition.PCA(n_components=5)
# pca.fit(df_proper_normalized)
# print(pca.explained_variance_ratio_)
# df_pca = pca.transform(df_proper_normalized)

In [31]:
def model_info2(model, train, test, y_train, y_test):
    g = y['hicov']
    skf = StratifiedKFold(g, 5)
    f1_scores = []
    auc_scores = []
    for train_index, test_index in skf:
        X_train, X_test = X.loc[train_index,:], X.loc[test_index,:]
        y_train, y_test = g[train_index], g[test_index]
        model.fit(X_train, y_train)
        digit_prec=2
        truthvec = y_test
        try:
            model.predict_proba
            scorevec = model.predict_proba(X_test)[:,1]
            threshvec = np.unique(np.round(scorevec,digit_prec))
            numthresh = len(threshvec)
            tpvec = np.zeros(numthresh)
            fpvec = np.zeros(numthresh)
            fnvec = np.zeros(numthresh)
            for i in range(numthresh):
                thresh = threshvec[i]
                tpvec[i] = sum(truthvec[scorevec>=thresh])
                fpvec[i] = sum(1-truthvec[scorevec>=thresh])
                fnvec[i] = sum(truthvec[scorevec<thresh])
            recallvec = tpvec/(tpvec + fnvec)
            precisionvec = tpvec/(tpvec + fpvec)
            f1vec = 2*precisionvec*recallvec/(precisionvec + recallvec)
            x = list(zip(threshvec, f1vec))
            rec_x = list(zip(threshvec, recallvec))
            max_f1 = sorted(x, key=lambda m: m[1], reverse=True)[0]
            max_f1_thresh = max_f1[0]
            recall_index = [i for i, v in enumerate(rec_x) if v[0] == max_f1_thresh][0]
            recall = rec_x[recall_index][1]
            print('Max F1 is %f which is for threshold %f' %(max_f1[1], max_f1_thresh))
            print('Corresponding recall is %f' %recall)
            f1_scores.append(max_f1[1])
        except:
            scorevec = model.predict(X_test)
            f1_scr = f1_score(y_test, scorevec)
            print('F1: ', f1_scr)
            f1_scores.append(f1_scr)
        auc = roc_auc_score(y_test, scorevec)
        auc_scores.append(auc)
        print('AUC: ', auc)
    mean_f1 = np.mean(f1_scores)
    mean_auc = np.mean(auc_scores)
    print('Mean F1: ', mean_f1)
    print('Mean AUC: ', mean_auc)
    col_keep = []
    col_reg = []
    try:
        model.feature_importances_
        features = list(zip(model.feature_importances_, X_train.columns))
        features.sort(key = lambda t: t[0], reverse=True)
        for i in range(len(features)):
            if features[i][0] != 0:
                col_keep.append(features[i][1])
                print('Importance: ', features[i])
            else:
                col_reg.append(features[i][1])
    except:
        print('Intercept: ', model.intercept_)
        zipped = sorted(list(zip(model.coef_[0], X.columns)), key= lambda t: abs(t[0]), reverse=True)
        for i in range(len(zipped)):
            if zipped[i][0] != 0:
                col_keep.append(zipped[i][1])
                print('Coefficient: ',zipped[i])
            else:
                col_reg.append(zipped[i][1])
    return model, col_keep, col_reg

In [39]:
with open('tfidf_final.pickle', 'wb') as handle:
    pickle.dump(tfidf_fit, handle)

In [40]:
with open('model_final.pickle', 'wb') as handle:
    pickle.dump(model, handle)

In [41]:
with open('tfidf_vectorizer_final.pickle', 'wb') as handle:
    pickle.dump(tfidf_vectorizer, handle)

In [ ]:
# df = pd.DataFrame(tfidf_matrix.todense(), columns=[tfidf_vectorizer.get_feature_names()])
# df['y'] = 0
# df['y'].ix[0:30999] = 1
# y = df[['y']]

In [32]:
def model_info(model, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    model.fit(X_train, y_train)
    digit_prec=2
    truthvec = y_test
    try:
        model.predict_proba
        scorevec = model.predict_proba(X_test)[:,1]
        threshvec = np.unique(np.round(scorevec,digit_prec))
        numthresh = len(threshvec)
        tpvec = np.zeros(numthresh)
        fpvec = np.zeros(numthresh)
        fnvec = np.zeros(numthresh)
        for i in range(numthresh):
            thresh = threshvec[i]
            tpvec[i] = sum(truthvec[scorevec>=thresh])
            fpvec[i] = sum(1-truthvec[scorevec>=thresh])
            fnvec[i] = sum(truthvec[scorevec<thresh])
        recallvec = tpvec/(tpvec + fnvec)
        precisionvec = tpvec/(tpvec + fpvec)
        f1vec = 2*precisionvec*recallvec/(precisionvec + recallvec)
        x = list(zip(threshvec, f1vec))
        rec_x = list(zip(threshvec, recallvec))
        max_f1 = sorted(x, key=lambda m: m[1], reverse=True)[0]
        #max_recall = sorted(rec_x, key=lambda m: m[1], reverse=True)[0]
        #fpr, tpr, thresholds = roc_curve(y_test, scorevec)
        #plt.plot(fpr, tpr);
        #plt.plot(fpr,fpr, 'r--');
        #plt.xlabel("False Positive Rate (1 - Specificity)");
        #plt.ylabel("True Positive Rate (Sensitivity)");
        #print(score)
        print('Max F1: ', max_f1)

    except:
        scorevec = model.predict(X_test)
        print('F1: ', f1_score(y_test, scorevec))
    score = roc_auc_score(y_test, scorevec)
#     x = list(zip(threshvec, f1vec))
#     rec_x = list(zip(threshvec, recallvec))
#     max_f1 = sorted(x, key=lambda m: m[1], reverse=True)[0]
#     #max_recall = sorted(rec_x, key=lambda m: m[1], reverse=True)[0]
#     fpr, tpr, thresholds = roc_curve(y_test, scorevec)
#     plt.plot(fpr, tpr);
#     plt.plot(fpr,fpr, 'r--');
#     plt.xlabel("False Positive Rate (1 - Specificity)");
#     plt.ylabel("True Positive Rate (Sensitivity)");
    print('AUC: ', score)
#     print('Max F1: ', max_f1)
#    print('Max Recall: ', max_recall)
    col_keep = []
    col_reg = []
#     try:
#         model.feature_importances_
#         features = list(zip(model.feature_importances_, X_train.columns))
#         features.sort(key = lambda t: t[0], reverse=True)
#         for i in range(len(features)):
#             if features[i][0] != 0:
#                 col_keep.append(features[i][1])
#             else:
#                 col_reg.append(features[i][1])
#     except:
#         zipped = sorted(list(zip(model.coef_[0], X.columns)), key= lambda t: abs(t[0]), reverse=True)
#         for i in range(len(zipped)):
#             if zipped[i][0] != 0:
#                 col_keep.append(zipped[i][1])
#             else:
#                 col_reg.append(zipped[i][1])
    return model, col_keep, col_reg

In [33]:
#df.head().iloc[:,:10]

In [ ]:
# train=df.sample(frac=0.8,random_state=200)
# test=df.drop(train.index)
# y_train = train['y']
# y_test = test['y']
# del train['y']
# del test['y']

In [34]:
model = LogisticRegression()
model.fit(train, y_train)
digit_prec=2
truthvec = y_test
try:
    model.predict_proba
    scorevec = model.predict_proba(test)[:,1]
    threshvec = np.unique(np.round(scorevec,digit_prec))
    numthresh = len(threshvec)
    tpvec = np.zeros(numthresh)
    fpvec = np.zeros(numthresh)
    fnvec = np.zeros(numthresh)
    for i in range(numthresh):
        thresh = threshvec[i]
        tpvec[i] = sum(truthvec[scorevec>=thresh])
        fpvec[i] = sum(1-truthvec[scorevec>=thresh])
        fnvec[i] = sum(truthvec[scorevec<thresh])
    recallvec = tpvec/(tpvec + fnvec)
    precisionvec = tpvec/(tpvec + fpvec)
    f1vec = 2*precisionvec*recallvec/(precisionvec + recallvec)
    x = list(zip(threshvec, f1vec))
    rec_x = list(zip(threshvec, recallvec))
    max_f1 = sorted(x, key=lambda m: m[1], reverse=True)[0]
    #max_recall = sorted(rec_x, key=lambda m: m[1], reverse=True)[0]
    #fpr, tpr, thresholds = roc_curve(y_test, scorevec)
    #plt.plot(fpr, tpr);
    #plt.plot(fpr,fpr, 'r--');
    #plt.xlabel("False Positive Rate (1 - Specificity)");
    #plt.ylabel("True Positive Rate (Sensitivity)");
    #print(score)
    print('Max F1: ', max_f1)

except:
    scorevec = model.predict(test)
    print('F1: ', f1_score(y_test, scorevec))
score = roc_auc_score(y_test, scorevec)
#     x = list(zip(threshvec, f1vec))
#     rec_x = list(zip(threshvec, recallvec))
#     max_f1 = sorted(x, key=lambda m: m[1], reverse=True)[0]
#     #max_recall = sorted(rec_x, key=lambda m: m[1], reverse=True)[0]
#     fpr, tpr, thresholds = roc_curve(y_test, scorevec)
#     plt.plot(fpr, tpr);
#     plt.plot(fpr,fpr, 'r--');
#     plt.xlabel("False Positive Rate (1 - Specificity)");
#     plt.ylabel("True Positive Rate (Sensitivity)");
print('AUC: ', score)

Max F1:  (0.40999999999999998, 0.9467005076142132)
AUC:  0.985138986084


/home/velocci/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning:

invalid value encountered in true_divide



In [ ]:
#print(rec_x)

In [ ]:
#log_reg, col_keep, col_reg = model_info2(LogisticRegression(penalty='l1', C=0.1), labeled_dummies, y)

In [ ]:
d2 = defaultdict(int)
#for i in range(1,51):
for j in range(31000):
    title = all_articles[0:31000][j]
    numbers = [int(s) for s in title.split() if s.isdigit()]
#        if str(i) in all_articles[0:31000][j].split():
    for number in numbers:
        d2[number] += 1
for k,v in d2.items():
    if k > 60:
        del d2[k]

In [ ]:
#d2_50 = sorted(d2.items(), key=operator.itemgetter(0))[1:51]

In [ ]:
del d2[0]

In [ ]:
keys = []
values = []
for k,v in d.items():
    keys.append(k)
    values.append(v)

In [ ]:
plt.bar(d2.keys(), d2.values(), color='g')

In [ ]:
bf = pd.read_csv('buzzfeed_facebook.csv')

In [ ]:
bf.columns

In [ ]:
del bf['title']

In [ ]:
bf.columns = ['Title', 'url', 'Author', 'Time', 'Category', 'Badge', 'Responses', 'FBShares', 'Tweets', 'FBComments']

In [ ]:
bf.head()

In [ ]:
bf['FBShares'] = bf['FBShares'].apply(lambda x: x.replace(',', '')).astype(int)

In [ ]:
bf_fb_info = dict()

In [ ]:
len(bf)

In [ ]:
for i in range(1,101):
    sum_occ = 0
    sum_shares = 0
    sum_tweets = 0
    sum_resp = 0
    sum_comm = 0
    for j in range(len(bf)):
        if str(i) in bf['Title'][j].split()[0:5]:
#        if str(i) + " " in bf['Title'][j][0:10]:
            sum_occ += 1
            sum_shares += bf['FBShares'][j]
            sum_tweets += bf['Tweets'][j]
            sum_resp += bf['Responses'][j]
            sum_comm += bf['FBComments'][j]
    if sum_occ > 0:
        avg_shares = sum_shares/sum_occ
        avg_tweets = sum_tweets/sum_occ
        avg_resp = sum_resp/sum_occ
        avg_comm = sum_comm/sum_occ
        bf_fb_info[i] = [sum_occ, sum_shares, sum_tweets, sum_resp, sum_comm, avg_shares, avg_tweets, avg_resp, avg_comm]

In [ ]:
shares = dict()
comments = dict()
responses = dict()
occurences = dict()
tweets = dict()
avg_shares = dict()
avg_comments = dict()
avg_responses = dict()
avg_tweets = dict()
for k,v in bf_fb_info.items():
    occurences[k] = v[0]
    shares[k] = v[1]
    tweets[k] = v[2]
    responses[k] = v[3]
    comments[k] = v[4]
    avg_shares[k] = v[5]
    avg_tweets[k] = v[6]
    avg_responses[k] = v[7]
    avg_comments[k] = v[8]

In [ ]:
plt.bar(shares.keys(), shares.values(), color='g')

In [ ]:
data = [go.Bar(
            x=d.keys(),
            y=d.values()
    )]
layout = go.Layout(
    #title='Buzzfeed List Number Occurrences - 2014 (Different Dataset)',
    xaxis=dict(
        title='List Number',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Frequency',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)
fig = go.Figure(data=data, layout=layout)
plot_url = py.plot(fig) #filename='styling-names')

In [ ]:
data = [go.Bar(
            x=shares.keys(),
            y=shares.values()
    )]

py.iplot(data) #filename='basic-bar')

In [ ]:
data = [go.Bar(
            x=avg_shares.keys(),
            y=avg_shares.values()
    )]
layout = go.Layout(
    title='Buzzfeed Average Article Facebook Shares - 2014 (Different Dataset)',
    xaxis=dict(
        title='List Number',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Avg Facebook Shares',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)
fig = go.Figure(data=data, layout=layout)
plot_url = py.plot(fig) #filename='styling-names')

In [ ]:
plt.bar(tweets.keys(), tweets.values(), color='g')

In [ ]:
data = [go.Bar(
            x=avg_tweets.keys(),
            y=avg_tweets.values()
    )]
layout = go.Layout(
    title='Buzzfeed Average Article Tweets - 2014 (Different Dataset)',
    xaxis=dict(
        title='List Number',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Avg Tweets',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)
fig = go.Figure(data=data, layout=layout)
plot_url = py.plot(fig) #filename='styling-names')

In [ ]:
data = [go.Bar(
            x=avg_tweets.keys(),
            y=avg_tweets.values()
    )]

py.iplot(data) #filename='basic-bar')

In [ ]:
plt.bar(occurences.keys(), occurences.values(), color='g')

In [ ]:
plt.bar(responses.keys(), responses.values(), color='g')

In [ ]:
plt.bar(comments.keys(), comments.values(), color='g')

In [ ]:
data = [go.Bar(
            x=avg_comments.keys(),
            y=avg_comments.values()
    )]
layout = go.Layout(
    title='Buzzfeed Average Facebook Comments - 2014 (Different Dataset)',
    xaxis=dict(
        title='List Number',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Avg Facebook Comments',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)
fig = go.Figure(data=data, layout=layout)
plot_url = py.plot(fig) #filename='styling-names')

In [ ]:
fb_shares = defaultdict(list)
for i in range(1,61):
    for j in range(len(bf)):
        if str(i) in bf['Title'][j].split():
            fb_shares[i].append(bf['FBShares'][j])

In [ ]:
#for i in range(1,61):
#    print (i, np.mean(fb_shares[i]))

In [ ]:
bf_authors = defaultdict(int)
for i in range(1,61):
    for j in range(len(bf)):
        if str(i) in bf['Title'][j].split():
            bf_authors[(bf['Author'][j],i)] += 1

In [ ]:
bf_authors

In [ ]:
author_bf = defaultdict(list)
for k,v in bf_authors.items():
    if " " in k[0]:
        author_bf[k[0]].append([k[1],v])

In [ ]:
author_bf

In [ ]:
author_total_articles = defaultdict(int)
for i in range(1,61):
    for j in range(len(bf)):
        if str(i) in bf['Title'][j].split() and ' ' in bf['Author'][j]:
            author_total_articles[bf['Author'][j]] += 1

In [ ]:
author_total_articles

In [ ]:
top20_author = sorted(author_total_articles.items(), key=operator.itemgetter(1), reverse=True)[:20]

In [ ]:
top20_author

In [ ]:
author_bf['Lauren Yapalater']

In [ ]:
name = 'Lauren Yapalater'
nums = []
totals = []
for entry in range(len(author_bf[name])):
    nums.append(author_bf[name][entry][0])
    totals.append(author_bf[name][entry][1])
plt.bar(nums, totals, color='g')

In [ ]:
name = 'Brian Galindo'
nums = []
totals = []
for entry in range(len(author_bf[name])):
    nums.append(author_bf[name][entry][0])
    totals.append(author_bf[name][entry][1])
plt.bar(nums, totals, color='g')

In [ ]:
name = 'Leonora Epstein'
nums = []
totals = []
for entry in range(len(author_bf[name])):
    nums.append(author_bf[name][entry][0])
    totals.append(author_bf[name][entry][1])
plt.bar(nums, totals, color='g')

In [ ]:
name = 'Alanna Okun'
nums = []
totals = []
for entry in range(len(author_bf[name])):
    nums.append(author_bf[name][entry][0])
    totals.append(author_bf[name][entry][1])
plt.bar(nums, totals, color='g')

In [ ]:
data = [go.Bar(
            x=totals,
            y=nums,
            orientation = 'h'
)]

py.iplot(data)

In [ ]:
author_boxplot_data = defaultdict(list)
for i in range(1,51):
    for j in range(len(bf)):
        if str(i) in bf['Title'][j].split() and ' ' in bf['Author'][j]:
            author_boxplot_data[bf['Author'][j]].append(i)

In [ ]:
author_boxplot_data

In [ ]:
articles_written = dict()
for k,v in author_boxplot_data.items():
    articles_written[k] = len(v)

In [ ]:
top20_authors = sorted(articles_written.items(), key=operator.itemgetter(1), reverse=True)[:20]

In [ ]:
top20_authors

In [ ]:
authors = []
top20_author_boxplot = author_boxplot_data.copy()
for entry in top20_authors:
    authors.append(entry[0])
for k,v in top20_author_boxplot.items():
    if k not in authors:
        del top20_author_boxplot[k]

In [ ]:
top20_author_boxplot

In [ ]:
author_list = []
boxplot_data = []
for k,v in top20_author_boxplot.items():
    author_list.append(k)
    boxplot_data.append(v)


In [ ]:
author_list

In [ ]:
boxplot_data

In [ ]:
y0 = boxplot_data[0]
y1 = boxplot_data[1]
y2 = boxplot_data[2]
y3 = boxplot_data[3]
y4 = boxplot_data[4]
y5 = boxplot_data[5]
y6 = boxplot_data[6]
y7 = boxplot_data[7]
y8 = boxplot_data[8]
y9 = boxplot_data[9]
y10 = boxplot_data[10]
y11 = boxplot_data[11]
y12 = boxplot_data[12]
y13 = boxplot_data[13]
y14 = boxplot_data[14]
y15 = boxplot_data[15]
y16 = boxplot_data[16]
y17 = boxplot_data[17]
y18 = boxplot_data[18]
y19 = boxplot_data[19]

trace0 = go.Box(
    y=y0
)
trace1 = go.Box(
    y=y1
)
trace2 = go.Box(
    y=y2
)
trace3 = go.Box(
    y=y3
)
trace4 = go.Box(
    y=y4
)
trace5 = go.Box(
    y=y5
)
trace6 = go.Box(
    y=y6
)
trace7 = go.Box(
    y=y7
)
trace8 = go.Box(
    y=y8
)
trace9 = go.Box(
    y=y9
)
trace10 = go.Box(
    y=y10
)
trace11 = go.Box(
    y=y11
)
trace12 = go.Box(
    y=y12
)
trace13 = go.Box(
    y=y13
)
trace14 = go.Box(
    y=y14
)
trace15 = go.Box(
    y=y15
)
trace16 = go.Box(
    y=y16
)
trace17 = go.Box(
    y=y17
)
trace18 = go.Box(
    y=y18
)
trace19 = go.Box(
    y=y19
)

data = [trace0, trace1, trace2, trace3, trace4, trace5, trace6, trace7, trace8, trace9, trace10, trace11, trace12, trace13, trace14, trace15, trace16, trace17, trace18, trace19]
py.iplot(data)

In [ ]:
fb_shares

In [ ]:
facebook_shares = []
list_lengths = []
for k,v in fb_shares.items():
    for shares in v:
        list_lengths.append(k)
        facebook_shares.append(shares)

In [ ]:
facebook_shares

In [ ]:
# Create a trace
trace = go.Scatter(
    x = list_lengths,
    y = facebook_shares,
    mode = 'markers'
)

data = [trace]

# Plot and embed in ipython notebook!
py.iplot(data) #,filename='basic-scatter')

# or plot with: plot_url = py.plot(data, filename='basic-line')

In [ ]:
log_facebook_shares = []
for i in facebook_shares:
    log_facebook_shares.append(np.log10(i))

In [ ]:
# Create a trace
trace = go.Scatter(
    x = list_lengths,
    y = log_facebook_shares,
    mode = 'markers'
)

data = [trace]

# Plot and embed in ipython notebook!
py.iplot(data) #,filename='basic-scatter')

# or plot with: plot_url = py.plot(data, filename='basic-line')

In [ ]:
avg_facebook_shares = []
list_lengths_foravg = []
for k,v in fb_shares.items():
    list_lengths_foravg.append(k)
    avg_facebook_shares.append(np.mean(v))

In [ ]:
trace = go.Scatter(
    x = list_lengths_foravg,
    y = avg_facebook_shares,
    mode = 'markers'
)

data = [trace]

# Plot and embed in ipython notebook!
py.iplot(data) #,filename='basic-scatter')

# or plot with: plot_url = py.plot(data, filename='basic-line')

In [ ]:
# Create a trace
trace = go.Scatter(
    x = np.log(list_lengths),
    y = np.log(facebook_shares),
    mode = 'markers'
)

data = [trace]

# Plot and embed in ipython notebook!
py.iplot(data) #,filename='basic-scatter')

# or plot with: plot_url = py.plot(data, filename='basic-line')

In [ ]:
# corpus = np.array(["aaa bbb ccc", "aaa bbb ddd"])
# vectorizer = CountVectorizer(decode_error="replace")
# vec_train = vectorizer.fit_transform(corpus)
# #Save vectorizer.vocabulary_
# pickle.dump(vectorizer.vocabulary_,open("feature.pkl","wb"))

# #Load it later
# transformer = TfidfTransformer()
# loaded_vec = CountVectorizer(decode_error="replace",vocabulary=pickle.load(open("feature.pkl", "rb")))
# tfidf = transformer.fit_transform(loaded_vec.fit_transform(np.array(["aaa ccc eee"])))

In [ ]:
test = tfidf_vectorizer.fit(all_articles)
test2 = test.transform(all_articles)

In [ ]:
test3 = test.transform(['Beyoncé Leads With Grammy Nominations'])
model.predict_proba(test3)
#model.predict(test3)

In [ ]:
all_articles[0]

In [ ]:
sum = 0
for i in range(31000):
    title = all_articles[i]
    numbers = [int(s) for s in title.split() if s.isdigit()]
    if len(numbers) > 0:
        sum += 1

In [ ]:
sum

In [ ]:
all_articles2 = []
for i in range(len(all_articles)):
    title = all_articles[i]
    numbers = [str(s) for s in title.split() if s.isdigit()]
    mapping = {}
    for number in numbers:
        mapping[number] = ''
    for k,v in mapping.iteritems():
        title = title.replace(k,v)
    all_articles2.append(title)

In [ ]:
tfidf_vectorizer = TfidfVectorizer(analyzer='word',
                                  ngram_range=(1, 3), stop_words=stop_words,
                                  min_df=3, max_df=1.0, max_features=None, strip_accents='unicode', token_pattern=r'\w{1,}',
                                  sublinear_tf=True)
tfidf_matrix = tfidf_vectorizer.fit_transform(all_articles2)

In [ ]:
df = pd.DataFrame(tfidf_matrix.todense(), columns=[tfidf_vectorizer.get_feature_names()])
df['y'] = 0
df['y'].ix[0:30999] = 1
y = df[['y']]

In [ ]:
train=df.sample(frac=0.8,random_state=200)
test=df.drop(train.index)
y_train = train['y']
y_test = test['y']
del train['y']
del test['y']

In [ ]:
model = LogisticRegression()
model.fit(train, y_train)
digit_prec=2
truthvec = y_test
try:
    model.predict_proba
    scorevec = model.predict_proba(test)[:,1]
    threshvec = np.unique(np.round(scorevec,digit_prec))
    numthresh = len(threshvec)
    tpvec = np.zeros(numthresh)
    fpvec = np.zeros(numthresh)
    fnvec = np.zeros(numthresh)
    for i in range(numthresh):
        thresh = threshvec[i]
        tpvec[i] = sum(truthvec[scorevec>=thresh])
        fpvec[i] = sum(1-truthvec[scorevec>=thresh])
        fnvec[i] = sum(truthvec[scorevec<thresh])
    recallvec = tpvec/(tpvec + fnvec)
    precisionvec = tpvec/(tpvec + fpvec)
    f1vec = 2*precisionvec*recallvec/(precisionvec + recallvec)
    x = list(zip(threshvec, f1vec))
    rec_x = list(zip(threshvec, recallvec))
    max_f1 = sorted(x, key=lambda m: m[1], reverse=True)[0]
    #max_recall = sorted(rec_x, key=lambda m: m[1], reverse=True)[0]
    #fpr, tpr, thresholds = roc_curve(y_test, scorevec)
    #plt.plot(fpr, tpr);
    #plt.plot(fpr,fpr, 'r--');
    #plt.xlabel("False Positive Rate (1 - Specificity)");
    #plt.ylabel("True Positive Rate (Sensitivity)");
    #print(score)
    print('Max F1: ', max_f1)

except:
    scorevec = model.predict(test)
    print('F1: ', f1_score(y_test, scorevec))
score = roc_auc_score(y_test, scorevec)
#     x = list(zip(threshvec, f1vec))
#     rec_x = list(zip(threshvec, recallvec))
#     max_f1 = sorted(x, key=lambda m: m[1], reverse=True)[0]
#     #max_recall = sorted(rec_x, key=lambda m: m[1], reverse=True)[0]
#     fpr, tpr, thresholds = roc_curve(y_test, scorevec)
#     plt.plot(fpr, tpr);
#     plt.plot(fpr,fpr, 'r--');
#     plt.xlabel("False Positive Rate (1 - Specificity)");
#     plt.ylabel("True Positive Rate (Sensitivity)");
print('AUC: ', score)

In [ ]:
print(rec_x)

In [ ]:
y_pred = model.predict(test)

print(accuracy_score(truthvec,y_pred))

In [ ]:
test = tfidf_vectorizer.fit(all_articles2)
test2 = test.transform(all_articles2)

In [ ]:
test3 = test.transform([all_articles[0]])
model.predict_proba(test3)

In [ ]:
all_articles[0]

In [ ]:
y_pred = model.predict(test)
print(accuracy_score(truthvec,y_pred))

text = bf_sort['Title'][37]
test3 = tfidf_fit.transform([text])
probs = model.predict_proba(test3)[0]

text = '"Everybody Loves Raymond" Star Doris Roberts Dies At Age 90'
test3 = tfidf_fit.transform([text])
model.predict_proba(test3)

In [ ]:
bf_sort = bf.sort(columns=['FBShares'], ascending=False)

In [ ]:
bf_sort = bf_sort.reset_index()

In [ ]:
bf_sort.head(50)

In [ ]:
file = open("buzzfeed.pickle",'r')
buzzfeed = pickle.load(file)

json_list = []
articles2 = []
for number in range(1,101):
    sum = 0
    articles = []
    for i in range(len(buzzfeed)):
        if str(number) in buzzfeed[i].split()[0:5]:
            articles.append({"t":buzzfeed[i], "l":number})
            sum += 1
    json_list.append({"y": sum, "x": number, "articles": articles})
    articles2.extend(articles)

In [ ]:
file = open("buzzfeed.pickle", 'r')
buzzfeed = pickle.load(file)
even_articles = []
odd_articles = []
for number in range(2,51,2):
    for i in range(len(buzzfeed)):
        if str(number) in buzzfeed[i].split()[0:5]:
            even_articles.append(buzzfeed[i])
for number in range(1,50,2):
    for i in range(len(buzzfeed)):
        if str(number) in buzzfeed[i].split()[0:5]:
            odd_articles.append(buzzfeed[i])

In [ ]:
file = open("buzzfeed.pickle",'r')
buzzfeed = pickle.load(file)
d = {}
for number in range(1,52):
    sum = 0
    for i in range(len(buzzfeed)):
        if str(number) in buzzfeed[i].split()[0:5]:
            sum += 1
    d[number] = sum


#numbers = [str(i) for i in range(1,52)]
#for title in buzzfeed:

In [ ]:
foo = tfidf_fit.transform(['Beyonce Leads with 9 grammy nominations'])
prob = model.predict_proba(foo)[0][1]

In [ ]:
print(prob)

In [ ]:
even_probs = []
for article in even_articles:
    foo = tfidf_fit.transform([article])
    even_probs.append(model.predict_proba(foo)[0][1])
print(np.mean(even_probs))

In [ ]:
odd_probs = []
for article in odd_articles:
    foo = tfidf_fit.transform([article])
    odd_probs.append(model.predict_proba(foo)[0][1])
print(np.mean(odd_probs))

In [ ]:
numbered_articles = even_articles + odd_articles
non_numbered_articles = []
for article in buzzfeed:
    if article not in numbered_articles:
        non_numbered_articles.append(article)

In [ ]:
non_numbered_probs = []
for article in non_numbered_articles:
    foo = tfidf_fit.transform([article])
    non_numbered_probs.append(model.predict_proba(foo)[0][1])
print(np.mean(non_numbered_probs))

In [ ]:
def CohenEffectSize(group1, group2):
    diff = np.mean(group1) - np.mean(group2)
    var1 = np.var(group1)
    var2 = np.var(group2)
    n1, n2 = len(group1), len(group2)
    pooled_var = (n1*var1 + n2*var2)/(n1 + n2)
    d = diff/math.sqrt(pooled_var)
    return d

In [ ]:
print(CohenEffectSize(odd_probs, even_probs))

In [ ]:
test = tfidf_vectorizer.fit(all_articles)
test2 = test.transform(all_articles)

In [ ]:
test3 = test.transform(['Beyoncé Leads With Grammy Nominations'])
model.predict_proba(test3)

In [ ]:
import json
with open('sample.json', 'w') as fp:
    json.dump(json_list, fp)